In [96]:
import yfinance as yf
import pandas as pd

ticker = "F"
ul_price = yf.Ticker(ticker).history(period='1d')['Close'][0]
expirations = yf.Ticker(ticker).options[:3]

calls = pd.concat( [yf.Ticker(ticker).option_chain(expirations[0]).calls, yf.Ticker(ticker).option_chain(expirations[1]).calls, yf.Ticker(ticker).option_chain(expirations[2]).calls])

calls['intrinsic_value'] = ul_price - calls['strike'] 
calls.loc[calls['intrinsic_value'] < 0, 'intrinsic_value'] = 0
calls['market_price'] = (calls['bid'] + calls['ask']) / 2
calls['extrinsic_value'] = calls['market_price'] - calls['intrinsic_value']
calls['extrinsic_to_strike_percentage'] = calls['extrinsic_value'] / calls['strike'] 
calls['extrinsic_to_underlying_percentage'] = calls['extrinsic_value'] / ul_price
calls['called_away_percentage'] = ( ul_price - calls['market_price'] - calls['strike']) / ul_price * -1

print(ul_price)
print(calls[calls['extrinsic_value'] > .1].to_markdown())

12.140000343322754
|    | contractSymbol   | lastTradeDate             |   strike |   lastPrice |   bid |   ask |   change |   percentChange |   volume |   openInterest |   impliedVolatility | inTheMoney   | contractSize   | currency   |   intrinsic_value |   market_price |   extrinsic_value |   extrinsic_to_strike_percentage |   extrinsic_to_underlying_percentage |   called_away_percentage |
|---:|:-----------------|:--------------------------|---------:|------------:|------:|------:|---------:|----------------:|---------:|---------------:|--------------------:|:-------------|:---------------|:-----------|------------------:|---------------:|------------------:|---------------------------------:|-------------------------------------:|-------------------------:|
|  0 | F230818C00003000 | 2023-06-13 14:01:43+00:00 |      3   |       11.1  | 12.1  | 12.15 |     0    |         0       |        1 |              3 |           43.5      | True         | REGULAR        | USD        |         

In [97]:
xxx = yf.Ticker(ticker).history(period='1d')['Close'][0]
print(xxx)

12.140000343322754


In [98]:
import json
print(json.dumps(yf.Ticker(ticker).info, indent=4) )

{
    "address1": "One American Road",
    "city": "Dearborn",
    "state": "MI",
    "zip": "48126-1899",
    "country": "United States",
    "phone": "313 322 3000",
    "website": "https://www.ford.com",
    "industry": "Auto Manufacturers",
    "industryDisp": "Auto Manufacturers",
    "sector": "Consumer Cyclical",
    "sectorDisp": "Consumer Cyclical",
    "longBusinessSummary": "Ford Motor Company develops, delivers, and services a range of Ford trucks, commercial cars and vans, sport utility vehicles, and Lincoln luxury vehicles worldwide. It operates through Ford Blue, Ford Model e, and Ford Pro; Ford Next; and Ford Credit segments. The company sells Ford and Lincoln vehicles, service parts, and accessories through distributors and dealers, as well as through dealerships to commercial fleet customers, daily rental car companies, and governments. It also engages in vehicle-related financing and leasing activities to and through automotive dealers. In addition, the company provi

In [99]:
import warnings
import datetime as dt
from pyblackscholesanalytics.market.market import MarketEnvironment
from pyblackscholesanalytics.options.options import PlainVanillaOption, DigitalOption

warnings.filterwarnings("ignore")

def option_factory(mkt_env, plain_or_digital, option_type, **kwargs):
    option_dispatcher = {
        "plain_vanilla": {"call": PlainVanillaOption(mkt_env, **kwargs),
                          "put": PlainVanillaOption(mkt_env, option_type="put", **kwargs)
                          },
        "digital": {"call": DigitalOption(mkt_env, **kwargs),
                    "put": DigitalOption(mkt_env, option_type="put", **kwargs)
                    }
    }

    return option_dispatcher[plain_or_digital][option_type]

def main():
    # if np_output is True, the output will be np.ndarray, otherwise pd.DataFrame
    np_output = False  # True

    # default market environment
    market_env = MarketEnvironment(dt.datetime.today(), 0.05, 3.39, 1.12)
    print(market_env)

     # define option style and type
    opt_style = "plain_vanilla"  # "digital" # "plain_vanilla"
    opt_type = "put"  # "put"
    option = option_factory(market_env, opt_style, opt_type, T='12-08-2023', K=3.5  )
    print(option)
    print(option.price())
    print(option.price_upper_limit())
    print(option.price_lower_limit())

main()

# Define the benchmark yield
bm_tickerSymbol = '^IRX'

# define the parameters as variables
S = 100 # underlying price
K = 110 # strike price
T = 0.5 # time to expiration (in years)
r = 0.05 # risk-free interest rate
q = 0 # dividend yield
sigma = 0.2 # volatility



Initializing the MarketEnvironment!
MarketEnvironment(t=12-08-2023, r=5.0%, S_t=3.4, sigma=112.0%)
Initializing the EuropeanOption!
Initializing the EuropeanOption!
Initializing the EuropeanOption!
Initializing the EuropeanOption!
PlainVanillaOption('put', S_t=3.4, K=3.5, t=12-08-2023, T=12-08-2023, tau=-0.00y, r=5.0%, sigma=112.0%)
[0.11]
[3.50047948]
[0.11047948]


In [100]:
import yfinance as yf
import pandas as pd
import datetime # import the datetime module

ticker = yf.Ticker("SPY") 
commish = 1
today = pd.to_datetime(datetime.date.today()) # get today's date as a DatetimeArray object


exp_dates = ticker.options # get all expiration dates
max_puts = [] # create an empty list to store the max_put for each date
today_data = ticker.history(period='1d') # get today's data
close_price = today_data['Close'].iloc[0] # get the close price
for date in exp_dates:
  opt = ticker.option_chain(date) # get option chain for a given date
  
  puts = opt.puts[(opt.puts['bid'] > 0) & (opt.puts['openInterest'] > 0)].copy() # get put options data and exclude puts that have a bid of 0
  
  puts.loc[:, 'lastTradeDate'] = pd.to_datetime(puts['lastTradeDate']) # convert the lastTradeDate column to a datetime object using .loc method
  puts.loc[:, 'lastTradeDate'] = puts['lastTradeDate'].dt.tz_localize(None) # remove the timezone information using .dt.tz_localize(None) method
  puts = puts[puts['lastTradeDate'] >= today - pd.Timedelta(days=7)] # filter out the puts that have not been traded in the last 30 days using .loc method


  # calculate extrinsic value and find the highest one
  puts.loc[:, 'mid'] = (puts['bid'] + puts['ask']) / 2 # calculate the mid point between the bid and ask using .loc method
  puts.loc[:, 'intrinsic'] = puts['strike'] - close_price # calculate intrinsic value using the close price using .loc method
  puts.loc[:, 'intrinsic'] = puts['intrinsic'].clip(lower=0) # ensure that intrinsic value is not less than 0 using .clip method
  puts.loc[:, 'extrinsic'] = puts['mid'] - puts['intrinsic'] # calculate extrinsic value using the mid point using .loc method


  max_extrinsic = puts['extrinsic'].max() # find the maximum extrinsic value
  max_put = puts[puts['extrinsic'] == max_extrinsic] # find the put option with the maximum extrinsic value
  max_puts.append(max_put) # add the max_put to the list


max_puts_df = pd.concat(max_puts) # concatenate the list of dataframes into a single dataframe

ticker_len = len(ticker.ticker) # get the length of the ticker symbol

max_puts_df.loc[:, 'expiryDate'] = pd.to_datetime(max_puts_df['contractSymbol'].str.slice(ticker_len, ticker_len+6), format='%y%m%d').dt.date # create a column named expiryDate using the contractSymbol column and convert it to a date object
max_puts_df.loc[:, 'dte'] = (pd.to_datetime(max_puts_df['expiryDate']) - today).dt.days # create a column named dte using the expiryDate column and calculate the days to expiration as an int

max_puts_df = max_puts_df[~(max_puts_df['dte'] == 0)] # remove rows where dte is 0

max_puts_df.loc[:, 'simpleReturn'] = (max_puts_df['extrinsic'] - commish/100) / max_puts_df['strike'] # calculate the simple return using the extrinsicValue and strike columns
max_puts_df.loc[:, 'annualized'] = max_puts_df['simpleReturn'] / max_puts_df['dte'] * 365  # calculate the annualized return using the extrinsicValue and strike columns

xxx_df = max_puts_df.loc[:, ['dte', 'expiryDate', 'strike', 'volume', 'openInterest', 'lastTradeDate', 'lastPrice', 'bid', 'ask', 'mid', 'extrinsic', 'simpleReturn', 'annualized', 'impliedVolatility']]
print(xxx_df.to_markdown())

#print(max_puts_df.to_markdown()) # print the updated dataframe as a markdown table


|     |   dte | expiryDate   |   strike |   volume |   openInterest | lastTradeDate       |   lastPrice |   bid |   ask |    mid |   extrinsic |   simpleReturn |   annualized |   impliedVolatility |
|----:|------:|:-------------|---------:|---------:|---------------:|:--------------------|------------:|------:|------:|-------:|------------:|---------------:|-------------:|--------------------:|
|  54 |     2 | 2023-08-14   |      446 |    78489 |           5259 | 2023-08-11 20:14:56 |        1.31 |  1.32 |  1.33 |  1.325 |    0.974994 |     0.00216366 |    0.394869  |           0.0710542 |
|  52 |     3 | 2023-08-15   |      446 |    16107 |           3050 | 2023-08-11 20:14:28 |        1.85 |  1.86 |  1.87 |  1.865 |    1.51499  |     0.00337443 |    0.410555  |           0.0907073 |
|  38 |     4 | 2023-08-16   |      446 |    16662 |           7201 | 2023-08-11 20:14:40 |        2.29 |  2.29 |  2.3  |  2.295 |    1.94499  |     0.00433855 |    0.395893  |           0.101877  |
|  57

In [101]:
import yfinance as yf
import pandas as pd
from IPython.display import HTML
import datetime

ticker = yf.Ticker("TQQQ") 
commish = 1
withinStrikePercentage = .05
today = pd.to_datetime(datetime.date.today())


exp_dates = ticker.options 
ticker_len = len(ticker.ticker)
max_puts = [] 
today_data = ticker.history(period='1d')
close_price = today_data['Close'].iloc[0] 
for date in exp_dates:
  opt = ticker.option_chain(date) 
  
  puts = opt.puts[(opt.puts['bid'] > 0) & (opt.puts['openInterest'] > 1000) & (opt.puts['strike'] >= (1-withinStrikePercentage) * close_price) & (opt.puts['strike'] <= (1+withinStrikePercentage) * close_price)].copy()
  
  puts.loc[:, 'lastTradeDate'] = pd.to_datetime(puts['lastTradeDate']) 
  puts.loc[:, 'lastTradeDate'] = puts['lastTradeDate'].dt.tz_localize(None) 
  puts = puts[puts['lastTradeDate'] >= today - pd.Timedelta(days=7)] 


  
  puts.loc[:, 'mid'] = (puts['bid'] + puts['ask']) / 2 
  puts.loc[:, 'intrinsic'] = puts['strike'] - close_price 
  puts.loc[:, 'intrinsic'] = puts['intrinsic'].clip(lower=0) 
  puts.loc[:, 'extrinsic'] = puts['mid'] - puts['intrinsic'] 

  puts.loc[:, 'expiryDate'] = pd.to_datetime(puts['contractSymbol'].str.slice(ticker_len, ticker_len+6), format='%y%m%d').dt.date 
  puts.loc[:, 'dte'] = (pd.to_datetime(puts['expiryDate']) - today).dt.days

  puts.loc[:, 'extrinsicPerDay'] = puts['extrinsic'] / puts['dte'] 

  puts.loc[:, 'exercisedReturn'] = (close_price - puts['strike'] + puts['extrinsic']- commish/100) / puts['strike']
  puts.loc[:, 'exercisedReturnAnnualized'] = puts['exercisedReturn'] / puts['dte'] * 365

  puts.loc[:, 'simpleReturn'] = (puts['extrinsic'] - commish/100) / puts['strike'] 
  puts.loc[:, 'annualized'] = puts['simpleReturn'] / puts['dte'] * 365  

  max_x = puts['exercisedReturn'].max() 
  max_put = puts[puts['exercisedReturn'] == max_x] 
  max_puts.append(max_put) 

  max_x = puts['extrinsic'].max() 
  max_put = puts[puts['extrinsic'] == max_x] 
  max_puts.append(max_put) 

  max_x = puts['simpleReturn'].max() 
  max_put = puts[puts['simpleReturn'] == max_x] 
  max_puts.append(max_put)

  max_x = puts['extrinsicPerDay'].max() 
  max_put = puts[puts['extrinsicPerDay'] == max_x] 
  max_puts.append(max_put)


max_puts_df = pd.concat(max_puts).sort_values(by='dte') 
max_puts_df = max_puts_df[~(max_puts_df['dte'] == 0)] 
max_puts_df = max_puts_df[~(max_puts_df['extrinsic'] < .1)] 

xxx_df = max_puts_df.loc[:, ['dte', 'expiryDate', 'strike', 'volume', 'openInterest', 'lastTradeDate', 'lastPrice', 'bid', 'ask', 'mid', 'extrinsic', 'extrinsicPerDay', 'exercisedReturn', 'exercisedReturnAnnualized', 'simpleReturn', 'annualized', 'impliedVolatility']]

xxx_df.drop_duplicates(subset=['dte', 'strike'], keep='first', inplace=True)

In [102]:
# Import the library
import jinja2
import pandas as pd

def toHtml(dataFrame, closing_price):
    # Convert the dataframe to a list of dictionaries
    data = dataFrame.to_dict(orient="records")
    
    # Create a html template with jinja2 syntax
    template = """
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #333;
                color: #fff;
            }
            table {
                border: 2px solid #fff;
                margin: 20px auto;
            }
            th, td {
                padding: 10px;
                border-bottom: 1px solid #ddd;
            }
            th {
                background-color: #f2f2f2;
                color: #000;
                text-align: left;
            }
            td {
                text-align: left;
                color: #fff;
            }
            
            .highlight>td {
                background-color: #FFCC00;
                color:#000
            }
        </style>
    </head>
    <body>
        <h1>Report</h1>
        <p>This is a report generated with jinja2 and xxx_df.</p>
        <table>
            <tr>
                <!-- Use the column names of your dataframe as headers -->
                {% for col in data[0].keys() %}
                <th>{{ col }}</th>
                {% endfor %}
            </tr>
            <!-- Loop through the rows of your dataframe -->
            {% for row in data %}
            <!-- Check if the strike is greater than closing_price -->
            {% if row.strike > closing_price %}
            <!-- Add the highlight class to the row -->
            <tr class="highlight">
            {% else %}
            <tr>
            {% endif %}
                <!-- Loop through the values of each row -->
                {% for val in row.values() %}
                <td>
                {% if val is number and val % 1 == 0 %}
                    {{ "%.0f" | format(val) }}
                {% elif val is number and val == "%0.2f"|format(val|float) %}
                    {{ "%.2f" | format(val) }}
                {% elif val is number and val % 1 != 0 %}
                    {{ "%.3f" | format(val) }}
                {% else %}
                    {{ val }}
                {% endif %}

                </td>
                {% endfor %}
            </tr>
            {% endfor %}
        </table>

    </body>
    </html>
    """

    # Load the template
    template = jinja2.Template(template)

    # Render the template with the data
    return template.render(data=data, closing_price=closing_price)

print(close_price)
#print(xxx_df.to_markdown())
report_html = toHtml(xxx_df, closing_price=close_price)
display(HTML(report_html))


39.09000015258789


dte,expiryDate,strike,volume,openInterest,lastTradeDate,lastPrice,bid,ask,mid,extrinsic,extrinsicPerDay,exercisedReturn,exercisedReturnAnnualized,simpleReturn,annualized,impliedVolatility
6,2023-08-18,38,5018,4435,2023-08-11 19:59:41,0.760,0.720,0.740,0.730,0.730,0.122,0.048,2.898,0.019,1.153,0.563
6,2023-08-18,39,5020,2535,2023-08-11 19:59:53,1.120,1.100,1.130,1.115,1.115,0.186,0.031,1.864,0.028,1.724,0.538
13,2023-08-25,37.500,5374,5720,2023-08-11 19:58:15,1.130,1.090,1.140,1.115,1.115,0.086,0.072,2.018,0.029,0.827,0.602
13,2023-08-25,39,2156,1299,2023-08-11 19:59:17,1.690,1.650,1.700,1.675,1.675,0.129,0.045,1.263,0.043,1.199,0.564
20,2023-09-01,40,114,1046,2023-08-11 19:49:24,2.510,2.490,2.590,2.540,1.630,0.082,0.018,0.324,0.041,0.739,0.543
34,2023-09-15,38,547,2753,2023-08-11 19:59:50,2.200,2.150,2.230,2.190,2.190,0.064,0.086,0.924,0.057,0.616,0.568
34,2023-09-15,39,198,1743,2023-08-11 19:30:14,2.630,2.560,2.640,2.600,2.600,0.076,0.069,0.738,0.066,0.713,0.549
125,2023-12-15,40,75,1714,2023-08-11 19:20:57,5.500,5.500,5.600,5.550,4.640,0.037,0.093,0.272,0.116,0.338,0.551
160,2024-01-19,37.500,20,1033,2023-08-11 13:47:55,5.200,4.900,5.150,5.025,5.025,0.031,0.176,0.402,0.134,0.305,0.574
160,2024-01-19,40,32,2997,2023-08-11 19:25:39,6.100,6.050,6.300,6.175,5.265,0.033,0.109,0.248,0.131,0.300,0.548
